In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset = pd.read_csv('insurance.csv')
len(dataset) #gives the number of entries in the table

dataset.head() #gives the first 5 rows of table

df = dataset
df["sex"] = pd.factorize(df['sex'])[0] #pd.factrorize can obtain a numerical expression for any value
df["smoker"] = pd.factorize(df["smoker"])[0] #The 0 is used as the starting value
df["region"] = pd.factorize(df["region"])[0]
df.head()

test_dataset = df.sample(frac=0.2)
train_dataset = df[~df.isin(test_dataset)].dropna() #isin is used to check is values are in df
(len(train_dataset)) #dropna is used to drop missing values

train_label = train_dataset.pop("expenses")
train_label.head()
test_label = test_dataset.pop("expenses")
test_label.head()

test_dataset.head()

normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

model = keras.Sequential([
    normalizer,
    layers.Dense(16),
    layers.Dense(32),
    layers.Dropout(.08),
    layers.Dense(1),
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse']
)
model.build()
model.summary()

history = model.fit(
    train_dataset,
    train_label,
    epochs=100,
    validation_split=0.5,
    verbose=0, # disable logging
)

print(history)



In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_label, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_label, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
